# Pruebas Neo4j

In [1]:
from neo4j import GraphDatabase as GD
import pandas as pd
import numpy as np

In [2]:
uri = 'bolt://localhost:7687'
user = 'neo4j'
psw = 'password'

### Primera prueba

In [14]:
d = GD.driver(uri, auth=(user, psw))

In [15]:
def get_pod(tx, cat):
    pods = []
    result = tx.run("MATCH (c:Category)<--(p) "
                   "WHERE c.name = $cat "
                   "RETURN p.id AS pod", cat=cat)
    for r in result:
        pods.append(r['pod'])
    return pods

In [16]:
with d.session() as sess:
    pod = sess.read_transaction(get_pod, "news")
    for p in pod:
        print(p)

e89a2cc24f1fa1138a56a532ce68ab4a
f5fce0325ac6a4bf5e191d6608b95797
f42e0606b820b99354ab58d423598493
e8be02e7d7281ba3edb77da6a9fe95cd
ebb2a803a79ea0b5abdefb92d1c816f5
fc32aac8fe942ed89feae0e164185912
c4eafea85540de95544b73777e19c696
b9a5920d121ff4e239e6ca3b210e65e7
a343c93505bffeaf20564bd2977b4743
f1a7cabd6127108563a0e38ae044b88a
fa7162cfb3963c4d51b67094a51740d6
c684ea6c24f8a6a16c18edd37f2aea41
fa3cac03e05c1e9ec792e95e4850d7ce


### Segunda Prueba
Generar una clase

In [61]:
class App:
    def __init__(self, uri, auth):
        self.driver = GD.driver(uri, auth = auth)
        
    def close(self):
        self.driver.close()
        
    def get_cat_pod(self, user_id):
        with self.driver.session() as sess:
            result = sess.write_transaction(
                self._get_cat_pod, user_id)
            return result
        
    @staticmethod
    def _get_cat_pod(tx, user_id):
        query = (
            "MATCH (u:User)-[r]->(Podcast)-->(Category)<--(p:Podcast) "
            "WHERE u.id = $user_id "
            "RETURN r, p "
        )
        result = tx.run(query, user_id=user_id)
        return pd.DataFrame([{'cum_rating':record['r']['rating'], 
                              'podcast_id':record['p']['id']}
                    for record in result])

In [62]:
app = App(uri, (user, psw))

In [63]:
df = app.get_cat_pod("F5B4FD0698E670E")
df.head()

,cum_rating,podcast_id
0,5,bead83f2330788f9e629cce951e1df99
1,5,cc451ec20dfe8f985c246cc71749917d
2,5,b16332f51b9746dc5a97c7ce37644fc6
3,5,c1adb6ca5ca39575420fda03c099b037
4,5,cc451ec20dfe8f985c246cc71749917d


In [64]:
df.groupby('podcast_id')['cum_rating'].sum().reset_index()

,podcast_id,cum_rating
0,a3a535f66c7e8004e7dc54c2b2829a9e,5
1,b16332f51b9746dc5a97c7ce37644fc6,55
2,bead83f2330788f9e629cce951e1df99,55
3,c1adb6ca5ca39575420fda03c099b037,55
4,c2adb71e6b51e1397b49e216e07f3cef,55
5,cc451ec20dfe8f985c246cc71749917d,555
6,ee41bdc529ac60b6cd3b9a50413b0dee,5
7,f335df99185d1401ada492230b42514a,5
8,fa3cac03e05c1e9ec792e95e4850d7ce,5


## Generar atributos para entrenar modelo

In [69]:
class GenAtr:
    def __init__(self, uri, auth):
        self.driver = GD.driver(uri, auth = auth)
        
    def close(self):
        self.driver.close()
        
    def get_cat_based(self, user_id, podcast_id):
        with self.driver.session() as sess:
            result = sess.write_transaction(
                self._get_cat_based, user_id, podcast_id)
            return result
        
    @staticmethod
    def _get_cat_based(tx, user_id, podcast_id):
        query = (
            "MATCH (u:User)-[r]->(Podcast)-->(Category)<--(p:Podcast) "
            "WHERE u.id = $user_id AND p.id = $podcast_id "
            "RETURN r"
        )
        result = tx.run(query, user_id=user_id, podcast_id=podcast_id)
        total = 0
        cnt = 0
        for rec in result:
            total += rec['r']['rating']
            cnt += 1
        return total, cnt
    
    def get_user_based(self, user_id, podcast_id):
        with self.driver.session() as sess:
            result = sess.write_transaction(
                self._get_user_based, user_id, podcast_id)
            return result
        
    @staticmethod
    def _get_user_based(tx, user_id, podcast_id):
        query = (
            "MATCH (u:User)-[r1]->(Podcast)<-[r2]->(User)-[r3]->(p:Podcast) "
            "WHERE u.id = $user_id AND p.id = $podcast_id "
            "RETURN r1.rating + r2.rating + r3.rating "
            "AS total"
        )
        result = tx.run(query, user_id=user_id, podcast_id=podcast_id)
        total = 0
        cnt = 0
        for rec in result:
            total += rec['total']
            cnt += 1
        return total, cnt
    

In [70]:
gen = GenAtr(uri, (user, psw))

Generación de atributos

In [77]:
gen.get_cat_based('96A3F0358E7808D', 'd34629baca14d87860276b12a6cb7a3b')

(10, 2)

In [78]:
gen.get_user_based('96A3F0358E7808D', 'd34629baca14d87860276b12a6cb7a3b')

(23, 2)